# 1. 构建训练数据集


In [ ]:
# 读取数据
'''
原始文本文件 `Lexis-Nexis_LRA.csv`，其中包含:
- 'Title': 新闻标题
- 'Source': 新闻来源
- 'Time': 新闻时间
- 'Year': 新闻年份
- 'Month': 新闻月份
- 'Day': 新闻日期
- 'Text': 新闻正文
'''
import pandas as pd
text_df = pd.read_csv('./data/Lexis-Nexis_LRA.csv')
text_df.head()

In [ ]:
# 添加ID列
text_df.insert(0, 'ID', range(len(text_df))) # 添加 id 列
text_df.head()

In [ ]:
'''
`Text`去除两端空格
去除`Year`、`Month`、`Day`列
就`Text`列进行编码与格式检查
就`Text`列进行去重
'''
cleaned_df = text_df.copy()
cleaned_df['Text'] = cleaned_df['Text'].str.strip()
cleaned_df.drop(columns=['Year', 'Month', 'Day'], inplace=True)
from tools import is_legal_text
cleaned_df = cleaned_df.loc[cleaned_df.Text.apply(is_legal_text)]
cleaned_df.drop_duplicates('Text', inplace=True)
# cleaned_df.to_csv('./data/cleaned.csv', index=False)
cleaned_df.head()

In [ ]:
'''
经处理数据 `cleaned.csv`，其中包含:
- 'ID': 原数据ID
- 'Title': 新闻标题
- 'Source': 新闻来源
- 'Time': 新闻时间
- 'Text': 新闻正文
'''
cleaned_df = pd.read_csv('./data/cleaned.csv')
from ast import literal_eval
cleaned_df['Text'] = cleaned_df.Text.apply(literal_eval)
cleaned_df.head()

In [ ]:
train_df = cleaned_df.copy()
train_df.head()

In [ ]:
'''
筛选出包含以下字段的文本片段：
- "Uganda"
- "Sudan"
- "Central African Republic", "CAR"
- "Democratic Republic of the Congo", "DRC"
'''
PF_targets = [
    ("Uganda",),
    ("Sudan",),
    ("Central African Republic", "CAR"),
    ("Democratic Republic of the Congo", "DRC",),
]

from tools import abstract_text
PF_matched = train_df.Text.apply(lambda s: abstract_text(s, PF_targets[0]))
for PF_target in PF_targets[1:]:
    PF_matched = map(lambda _: list(map(any, zip(_[0], _[1]))), zip(PF_matched, train_df.Text.apply(lambda s: abstract_text(s, PF_target))))
train_df['PF_TARGET'] = list(PF_matched)
train_df.head()

In [ ]:
'''
筛选出包含以下字段的文本片段：
- "Lord's Resistance Army", "LRA"
'''
LRA_targets = [
    ("Lord's Resistance Army", "LRA",),
]

LRA_matched = train_df.Text.apply(lambda s: abstract_text(s, LRA_targets[0]))
for LRA_target in LRA_targets[1:]:
    LRA_matched = map(lambda _: list(map(any, zip(_[0], _[1]))), zip(LRA_matched, train_df.Text.apply(lambda s: abstract_text(s, LRA_target))))
train_df['LRA_TARGET'] = list(LRA_matched)
train_df.head()

In [ ]:
# train_df.to_csv('./data/train.csv', index=False)

In [ ]:
'''
筛选出同时包含PF和LRA目标的300个文本片段
'''
size = 300
target_indices = train_df.PF_TARGET.apply(lambda _: any(_)) & train_df.LRA_TARGET.apply(lambda _: any(_))
import numpy as np
target_df = train_df.loc[target_indices, ['ID', 'Text']].iloc[np.linspace(0, target_indices.sum(), num=300, endpoint=False, dtype=int)]
target_df[['ASK', 'ANSWER']] = pd.DataFrame(columns=['ASK', 'ANSWER'])
target_df.head()


In [ ]:
'''
消息模板
'''
ASK_TEMPLATE = """
{}

请根据以下六个基本标准，对上文进行0-1标注

基本标准：
(1)“PF_score”，报告提到伙伴部队的成功
(2)“PF_US”，报告提到美国与伙伴部队合作
(3)“PF_neg”，报告美国伙伴部队的负面情况
(4)“Threat_up”，报告提到上帝抵抗军的威胁增加
(5)“Threat_down”，上帝抵抗军的威胁是否减少
(6)“Citizen_impact”，公民是否受到上帝抵抗军暴力的影响
"""

In [ ]:
# # 尝试标注
# _id, _text, *_ = target_df.loc[target_df.ANSWER.isna()].iloc[0]
# ask = ASK_TEMPLATE.format(_text)
# answer = ask_deepseek(ask)
# target_df.loc[target_df.index[0], ['ASK', 'ANSWER']] = [ask, answer]
# target_df.head()

In [ ]:
# # 进行标注
# for _index, _id, _text, *_ in tqdm(list(target_df.loc[target_df.ANSWER.isna()].itertuples())):
#     ask = ASK_TEMPLATE.format(_text)
#     answer = ask_deepseek(ask)
#     target_df.loc[_index, ['ASK', 'ANSWER']] = [ask, answer]
# target_df.head()

In [ ]:
# # 提取标注结果
# import re
# _target_dict = target_df.ANSWER.apply(re.compile("([PFThreatCitizen]+_[\w]+)[^\d]+?(\d)").findall).apply(lambda _: {key: value for key, value in _[:6]}).to_dict()
# len(_target_dict)

In [ ]:
# _target_df = pd.DataFrame.from_dict(
#     _target_dict, orient='index'
# )
# _target_df.head()

In [ ]:
# target_df[_target_df.columns] = _target_df
# target_df.head()

In [ ]:
# target_df.to_csv('./data/target_df.csv', index=False)

In [ ]:
# TARGET_MD = """ # ID-{}.
# - ASK:

# ```
# {}

# 请根据以下六个基本标准，对上文进行0-1标注

# 基本标准：
# (1)“PF_score”，报告提到伙伴部队的成功
# (2)“PF_US”，报告提到美国与伙伴部队合作
# (3)“PF_neg”，报告美国伙伴部队的负面情况
# (4)“Threat_up”，报告提到上帝抵抗军的威胁增加
# (5)“Threat_down”，上帝抵抗军的威胁是否减少
# (6)“Citizen_impact”，公民是否受到上帝抵抗军暴力的影响
# ```

# - ANSWER:

# ```
# {}
# ```
# ---
# """

In [ ]:
# from itertools import chain
# with open('./data/target_df.md', 'w') as f:
#     f.write((TARGET_MD * len(target_df)).format(*chain(*zip(target_df.ID, target_df.Text, target_df.ANSWER))))

In [ ]:
# train_target_df = train_df.merge(target_df[['ID', 'PF_score', 'PF_US', 'PF_neg', 'Threat_up', 'Threat_down', 'Citizen_impact']], on='ID', how='right')
# train_target_df.head()

In [ ]:
# train_target_df.to_csv('./data/train_target_df.csv', index=False)

# 2. 模型 & 训练

In [ ]:
# ! nohup python main.py > ./data/log.log 2>&1 &

# 3. 获取评估结果

In [ ]:
import torch
from model.text_assessment.text_assessor import TextAssessor
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TextAssessor.loads(save_model='./data/text_assessor_B(350).pth', device=device)
print(model, device)

In [ ]:
from model.text_assessment import PRETRAINED_MODEL_NAME
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
print(tokenizer)


In [ ]:
import pandas as pd
data_df = pd.read_csv('./data/cleaned.csv')
from ast import literal_eval
data_df['Text'] = data_df.Text.apply(literal_eval)
data_df.head()

In [ ]:
from model.text_assessment.text_assessor import predict_one
from model.text_assessment import MAX_LEN
from tqdm.auto import tqdm
# 这里发现，数据中存在Text列为空列表的情况
items = {}
for _id, *_, _text in tqdm(data_df.itertuples(index=False), total=len(data_df)):
    pred = predict_one(data_text=_text, model=model, tokenizer=tokenizer, max_len=MAX_LEN, device=device) \
        if _text else {
        "pred.values.PF_score": None,
        "pred.values.PF_US": None,
        "pred.values.PF_neg": None,
        "pred.values.Threat_up": None,
        "pred.values.Threat_down": None,
        "pred.values.Citizen_impact": None,
    }
    items[_id] = pred


In [11]:
items_df = pd.DataFrame.from_dict(items, orient='index')
items_df.head()

,pred.values.PF_score,pred.values.PF_US,pred.values.PF_neg,pred.values.Threat_up,pred.values.Threat_down,pred.values.Citizen_impact
0,0.059625,0.060334,0.023490,0.281067,0.254386,0.714013
1,0.387642,0.747410,0.197883,0.498937,0.353932,0.665519
3,0.108162,0.330988,0.062792,0.197920,0.253518,0.698617
4,0.090984,0.079802,0.033550,0.297495,0.317629,0.611865
6,0.072882,0.042844,0.023981,0.372860,0.277219,0.714023


In [17]:
result_df = data_df.merge(items_df, left_on='ID', right_index=True, how='left')
result_df.head()

,ID,Title,Source,Time,Text,pred.values.PF_score,pred.values.PF_US,pred.values.PF_neg,pred.values.Threat_up,pred.values.Threat_down,pred.values.Citizen_impact
0,0,War criminal walking around in plain sight,The Calgary Herald (Alberta),"March 19, 2012 Monday",[The International Criminal Court has reached ...,0.059625,0.060334,0.023490,0.281067,0.254386,0.714013
1,1,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST","[Geneva, Mar 31 (IBNS) The United Nations refu...",0.387642,0.747410,0.197883,0.498937,0.353932,0.665519
2,3,Review: COVER STORY: KONY 2012: What happened ...,The Observer (London),15-Jul-12,"[For 25 years, Joseph Kony and his Lord's Resi...",0.108162,0.330988,0.062792,0.197920,0.253518,0.698617
3,4,Uganda 's war-scarred north set for peacetime ...,Agence France Presse -- English,"February 15, 2011 Tuesday 9:21 AM GMT",[Displaced northern Ugandans have flocked home...,0.090984,0.079802,0.033550,0.297495,0.317629,0.611865
4,6,BRITISH SAFARI MAN ACCUSED OF MASS MURDER 'FRAMED,MAIL ON SUNDAY (London),"May 13, 2012 Sunday","[LOCAL POACHERS', FROM TIMOTHY EVANS IN BANGUI...",0.072882,0.042844,0.023981,0.372860,0.277219,0.714023


In [18]:
# result_df.to_csv('./data/result_df.csv', index=False)